![Doggos-101](https://i.ibb.co/n7zX33C/doggos-loggos-nb-txt.png)

## Imports

In [ ]:
import os,sys
import cv2
from tqdm import tqdm
import re
import random

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical, image_dataset_from_directory

# inception_v3 import
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_input_incv3, InceptionV3
from tensorflow.keras.applications.efficientnet import EfficientNetB3

#extra
from tensorflow.keras.layers import Input

# Try Models

## Load and preprocess data using image_dataset_from_directory for better performance

In [ ]:
# Use this to see what directories you have:
# for root, folders, filenames in os.walk('/kaggle/input'):
#    print(root, folders)

# path for kaggle train/test (data should be train/test or train/val/test splitted)
train_dir = '/kaggle/input/cropped/cropped/train' # use paths to your data location
test_dir = '/kaggle/input/cropped/cropped/train' # use paths to your data location

In [ ]:
# load train
train_ds = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# preprocess train
prep_train_ds = train_ds.map(lambda x, y: (preprocess_input_incv3(x), y))

In [ ]:
# load val
validation_ds = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# preprocess val
prep_val_ds = validation_ds.map(lambda x, y: (preprocess_input_incv3(x), y))

In [ ]:
# load test
test_ds = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    batch_size=32)

# preprocess test
prep_test_ds = test_ds.map(lambda x, y: (preprocess_input_incv3(x), y))

## Load and freeze your model

In [ ]:
# specify the shape for the input layer
input_shape = train_ds.element_spec[0].shape[1:] #(224, 224, 3)
input_tensor = Input(shape=input_shape)

In [ ]:
# inception is a base model
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)
# freeze the base model layers to prevent weights from being updated during training
base_model.trainable = False

## Combine base model with own dense layers

In [ ]:
def model_init():
    augmentation = Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomZoom(0.1),
        layers.RandomTranslation(0.2, 0.2),
        layers.RandomRotation(0.1)
    ])

    model = Sequential([
        layers.Input(shape = input_shape),
        augmentation,
        base_model,
        layers.Flatten(),
        layers.Dense(100, activation="relu"),
        layers.Dense(100, activation="relu"),
        layers.Dense(120, activation='softmax')
    ])
    return model

In [ ]:
inception_model = model_init()
inception_model.summary()

In [ ]:
opt = optimizers.Adam(learning_rate=1e-4)

def compile_model(model):
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
inception_1 = compile_model(inception_model) # always call compile_model in the same cell as your model.fit

MODEL = "name" # will be in the name of your .h5 file

epochs = 20

es = EarlyStopping(monitor='val_loss', 
                   mode='auto', 
                   patience=3, 
                   verbose=1, 
                   restore_best_weights=True)

lr = ReduceLROnPlateau(monitor="val_loss",
                       factor=0.1,
                       patience=2,
                       verbose=1,
                       min_lr=0)

mcp = ModelCheckpoint(f"{MODEL}.h5",
                      save_weights_only=True,
                      monitor='val_accuracy',
                      mode='max',
                      verbose=0,
                      save_best_only=True)

history_1 = inception_1.fit(prep_train_ds, 
                        validation_data=prep_val_ds, 
                        epochs=epochs,
                        callbacks=[es, lr, mcp],
                        batch_size=32,
                        verbose=1)

In [ ]:
# score model
loss, accuracy_1 = inception_1.evaluate(prep_test_ds)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy_1:.4f}")

In [ ]:
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [ ]:
plot_history(history_1)